<img src="https://github.com/CorndelDataAnalyticsDiploma/workshop/blob/master/Corndel%20Digital%20Logo%20Centre.png?raw=true" alt="Corndel" width ="301.5" align="left">

# Predictive Modelling

In this notebook we will build a Linear Predictive Model.

## The Statisticians' Approach vs the Machine Learning Approach

### The Statisticians' Approach (Recap)

In the first section we built a linear model using the traditional statisticians' approach, whose focus was on **gaining understanding about the relationships between the dependent and independent variables** e.g. how much do the independent variables impact on the dependent variable?

This approach is predicated on **a number of assumptions** about the data (e.g. linearity, a normal distribution of residuals and a lack of collinearity between the independent variables)

### The Machine Learning Approach

The focus of the Machine Learning (ML) approach is on **making predictions** i.e. how good is the model at making predictions?

It still makes assumptions, but these are **less strict**.

We determine how good the model is by **evaluating it against data it has not seen before**. How well does the model generalise to new data?


### Simple Linear Regression

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error


# Multiple Linear Regression

We will start with Simple Linear Regression (i.e. 1 independent variable) and gradually build in more independent variables. This is called **forward selection**.

This is a very simple method of working out which combination of independent variables we should use in the model. 



Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

Number of rows: 442

Number of columns: First 10 columns are numeric predictive values

Target: Column 11 (`Y`) is a quantitative measure of disease progression one year after baseline.

Attribute Information:
- Age
- Sex
- BMI - Body mass index
- BP - average blood pressure
- S1...S6 - blood serum measurements 1-6

Source URL:
http://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

For more information see:
Bradley Efron, Trevor Hastie, Iain Johnstone and Robert Tibshirani (2004) "Least Angle Regression," Annals of Statistics (with discussion), 407-499.
(http://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf)

In [ ]:
# Load the diabetes dataset and display the head
df_dia = pd.read_csv("https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt",sep="\t")
df_dia.head()

<div class="alert alert-block alert-warning">

#### Any issues with this data?
    
    --Encoding of Sex is an issue
    
    -- We will address later but discuss now

If you wish to practice encoding categorical variables, this is covered is covered in the extension `Encoding.ipynb` notebook.

# Discussion on encoding

Categorical encoding, ordinal, one-hot encoding.

🤖`*"can i encode male and female as 1 and 2 for linear regression model?"*`

An AI Chatbot will warn about issues with this and offer solutions.

## Step 1 (EDA)


<div class="alert alert-block alert-warning">

Examine the data. Does it look like there is a linear relationship?

Is a linear model suitable?

Do you see any outliers?

What options do you have to deal with outliers?

In [ ]:
# Get a description of the data
df_dia.describe().round()

In [ ]:
# Use Pandas to generate histograms of all columns, bins = 50 and figsize= (20,20) 
# note: no easy way to do this in Seaborn

df_dia.hist(figsize = (20,20),bins = 50)
plt.show()

<div class="alert alert-block alert-warning">

# Obervations
You will notice that BMI, S3 and the dependent variable (`Y`) are all skewed.\
Transformations should be applied to these columns.\
This is covered in the extension material.

# Pairplot

In [ ]:
# Use Seaborn to generate Pair Plots

sns.pairplot(df_dia)
plt.show()

<div class="alert alert-block alert-warning">

# Observations
We can see S1 and S2 are highly correlated (see centre of grid). 

This would be an issue for Regression Analysis, but does not concern us for predictive modelling.

Let's plot a candidate independent variable (`S5`) against the dependent variable (`Y`).

In [ ]:
sns.scatterplot(data=df_dia, x="S5", y="Y")
plt.show()

This doesn't look hugely linear but let's calculate its correlation coefficient (r).

In [ ]:
# Calculate correlation coefficient and its p-value
correlation, p_value = stats.pearsonr(df_dia['S5'], df_dia['Y'])

print(f"Pearson correlation coefficient: {round(correlation, 2)}")
print(f"P-value: {round(p_value, 5)}")

# Train, Test Split

We must divide the data into:
 - a Training set and 
  - a Test set 
  - with a 70:30 split
  
We will build the model using the training data and asses how well our model performs on the test data.\
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

To test assess how well our model performs we will use both:
 - Root Mean Squared Error (RMSE) and 
  - Mean Absolute Percentage Error (MAPE). 
  
 We will look at these in detail later.

In [ ]:
# Examine columns again

df_dia.columns

In [ ]:
# Assign Inputs to X and output to y. I copied and pasted these from the above output.

X = df_dia[['AGE', 'SEX', 'BMI', 'BP', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']]
y = df_dia['Y']

In [ ]:
# Use Scikit-Learn to split the data into training and test sets
# By default, train_test_split() does random sampling with shuffling.
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.30, 
                                                    random_state=42)

In [ ]:
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

<div class="alert alert-block alert-warning">

# Discussion


<img src="assets/TrainTestDiagram.png" alt="Corndel" width ="600" align="centre">

As you can see from the diagram above, the approach is to:

1) build a model using the training data
2) try to build a better model using the training data (and repeat)
3) pick the best model and evaluate it using the test data (to see how well it performs on 'unknown' data).


Examining the test and train sets.

Discussion about stratfied samples.

<img src="assets/Stratified.jpg" alt="Corndel" width ="600" align="centre">

In [ ]:
# Numerically examine the X_train split

X_train.describe().round(2)

In [ ]:
# Numerically examine the X_test split
X_test.describe().round(2)

<div class="alert alert-block alert-warning">

# Discussion
    What do you notice about the two samples?
    In which ways are they similar?
    In which ways are they dissimilar?

In [ ]:
# Numerically examine the y_train split
# (to_frame() just converts a pandas Series into a DataFrame).

y_train.to_frame().describe().round(2)

In [ ]:
# Numerically examine the y_test split

y_test.to_frame().describe().round(2)

# Simple Linear Regression

### We will build our first model using the S5 marker

In [ ]:
# Use Seaborn to create a scatter plot of S5 against Y

sns.scatterplot(data=df_dia, x="S5", y="Y")
plt.show()

Let's fit the model and make predictions.

You will note that we do not print out a model summary anymore. In this instance we are not concered about estimating coefficients and the statistical significance of these estimates.  

Our focus is **how well we can make predictions**.

In [ ]:
# Selected a single independent variable
X = X_train['S5']
# Add a constant
X = sm.add_constant(X)

y = y_train

# Fit model
model = sm.OLS(y, X).fit()

# make predictions
#pred = model.predict(sm.add_constant(X_test['S5']))
pred = model.predict(sm.add_constant(X_train['S5']))

### Examine the output, look at the predictions and the test set

Notice how the indexes line up - now we can get metrics for our results.

In [ ]:
# examine predictions
pred

In [ ]:
# examine true values
#y_test
y_train

<div class="alert alert-block alert-warning">

# Discussion
    What do you notice about the two samples?
    In which ways are they similar?
    In which ways are they dissimilar?

## Root Mean Squared Error (RMSE)

An *error* is the difference between an *actual value* and a *predicted value*. It doesn't mean "mistake". 😉

RMSE measures how many "units" of the dependent variable we are off, on average.
The "Root Mean Squared" ensures it is always positive.

A **lower** RMSE value is better.

https://en.wikipedia.org/wiki/Root_mean_square_deviation

In [ ]:
# Using Scikit-Learn Package to calculate RMSE
# rmse = root_mean_squared_error(y_test, pred).round(2)
rmse = root_mean_squared_error(y_train, pred).round(2)
print(f'RMSE: {rmse}')

## Mean Absolute Percentage Error (MAPE)
This measures what *percentage* we are off on average.\
The "Absolute" ensures it is always positive.

A **lower** MAPE value is better.

https://en.wikipedia.org/wiki/Mean_absolute_percentage_error

In [ ]:
# Using Scikit-Learn Package to calculate RMSE
#mape = mean_absolute_percentage_error(y_test, pred).round(2)*100
mape = mean_absolute_percentage_error(y_train, pred).round(2)*100
print(f'MAPE: {mape}%')

# Multiple Linear Regression
Lets now add in some independent variables and see how our results change.

In [ ]:
# Look at the head of the training independent variables again

X_train.head()

Lets add in BMI.

In [ ]:
# Create a list called "inputs" with "S5" and "BMI" as elements. 
# Note notation for lists is []

inputs = ['S5','BMI']

In [ ]:
# Fit the model with both inputs, and calculte the new RMSE and MAPE Scores

X = sm.add_constant(X_train[inputs])  # adds intercept term
y = y_train

# Fit model
model = sm.OLS(y, X).fit()

# make predictions
#pred = model.predict(sm.add_constant(X_test[inputs]))
pred = model.predict(sm.add_constant(X_train[inputs]))

#rmse_2 = root_mean_squared_error(y_test, pred).round(2)
rmse_2 = root_mean_squared_error(y_train, pred).round(2)
print(f'RMSE_2: {rmse_2}')

#mape_2 = mean_absolute_percentage_error(y_test, pred).round(2)*100
mape_2 = mean_absolute_percentage_error(y_train, pred).round(2)*100
print(f'MAPE_2: {mape_2}')

In [ ]:
# Compare these new scores to the previous
#rmse_2 = root_mean_squared_error(y_test, pred).round(2)
rmse_2 = root_mean_squared_error(y_train, pred).round(2)
print(f'RMSE_2: {rmse_2}, Previous RMSE: {rmse}, Improvement = {(rmse - rmse_2).round(2)}' )

#mape_2 = mean_absolute_percentage_error(y_test, pred).round(2)*100
mape_2 = mean_absolute_percentage_error(y_train, pred).round(2)*100
print(f'MAPE_2: {mape_2}%  Previous MAPE: {mape}%, Improvement = {(mape - mape_2).round(2)}%')

<div class="alert alert-block alert-warning">

# Observations
    We can see here that we have made a marginal improvement by adding BMI.

Let's say that we have built all the models we want to (all 2 of them!) and are ready to test our best model (i.e. the second one) with unknown data.

In [ ]:
# make predictions
inputs = ['S5','BMI']
pred = model.predict(sm.add_constant(X_test[inputs]))

rmse_2 = root_mean_squared_error(y_test, pred).round(2)
print(f'RMSE_2: {rmse_2}')

mape_2 = mean_absolute_percentage_error(y_test, pred).round(2)*100
print(f'MAPE_2: {mape_2}')

This is the end of our model-building and evaluation. 

# Menti for Quiz on Terms

# Fix the SEX column

In [ ]:
# Encode using one-hot encoding
df_dia = pd.get_dummies(df_dia, columns=['SEX'], drop_first=True, dtype=int)

df_dia

# Explore adding more independent variables

Use the code below as a template to add more features and evaluate any changes in output.

A good idea would be to save the metrics as element in a dictionary to compare results.

In [ ]:
# Creating a dict to hold results
# You can add the results for each model to it
results = {}

In [ ]:
# look at features
X_train.head()

## Your Turn

Run the two cells below repeatedly with different combinations of independent variables then examine the dictionary of stored results.

In [ ]:
# Select features

inputs = [.......]

In [ ]:
# Fit the model with both inputs, and calculte the new RMSE and MAPE Scores

X = sm.add_constant(X_train[inputs])  # adds intercept term
y = y_train

# Fit model
model = sm.OLS(y, X).fit()

# make predictions
pred = model.predict(sm.add_constant(X_test[inputs]))


# be aware of overwriting previous results, by calling this variable "rmse" we overwrite what was previously stored 
rmse = root_mean_squared_error(y_test, pred).round(2)
# printing results
print(f'RMSE: {rmse}')

# be aware of overwriting previous results, by calling this variable "mape" we overwrite what was previously stored 
mape = mean_absolute_percentage_error(y_test, pred).round(2)*100
# printing results
print(f'MAPE: {mape}')

# adding result to dict
results[str(inputs)] = {'RMSE':rmse, "MAPE": mape}


In [ ]:
# Printing out dict of results
results